In [ ]:
from unsloth import FastLanguageModel, unsloth_train
# import os
import json
import torch
import random
import numpy as np
import pandas as pd
import datasets
from datasets import Dataset, load_dataset
# from transformers import (
#     # AutoConfig, AutoModelForCausalLM, AutoTokenizer,
#     # Seq2SeqTrainer, Seq2SeqTrainingArguments
# )
# from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
import bitsandbytes as bnb
# from transformers import BitsAndBytesConfig
# from peft import PeftModel, PeftConfig

from huggingface_hub import login
import wandb

# Load API tokens
CONFIG = json.load(open('/home/jupyter/datasphere/project/tokens.json'))
login(token=CONFIG["HF_TOK"])
wandb.login(key=CONFIG['WANDB_API_KEY'])

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

def set_random_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_random_seed()

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-06-01 01:11:35.516589: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-06-01 01:11:39.642302: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


🦥 Unsloth Zoo will now patch everything to make training faster!


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rostik58533 (mika5883). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
# Load dataset
dataset = load_dataset("csv", data_files=["/home/jupyter/datasphere/project/rugec/data/art_gec_full.tsv"], sep="\t")
dataset = dataset["train"].rename_columns({'correct': 'correct_sent', 'corrupt': 'corrupt_sent'})
# dataset = dataset.filter(lambda example: example['sentence1'].startswith('Ar'))
# Load additional datasets
def load_tsv(path):
    return pd.read_csv(path, sep="\t", index_col=None)

dev = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.dev.tsv')
train = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.train.tsv')
test = load_tsv('/home/jupyter/datasphere/project/rugec/data/RULEC-GEC.test.tsv')
clang8 = load_tsv('/home/jupyter/datasphere/project/rugec/data/clang8_source_target_ru.spacy_tokenized.tsv')
relco = load_tsv('/home/jupyter/datasphere/project/rugec/data/relco_filtered.tsv')
gera_train = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.train.tsv')
# gera_test = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.test.tsv')
gera_dev = load_tsv('/home/jupyter/datasphere/project/rugec/data/GERA.dev.tsv')
my_data = load_tsv('/home/jupyter/datasphere/project/rugec/data/25k_NVP.tsv')
# train.corrupt_sent = train.corrupt_sent.map(lambda x: str(x))
# test.corrupt_sent = test.corrupt_sent.map(lambda x: str(x))
# dev.corrupt_sent = dev.corrupt_sent.map(lambda x: str(x))
# train.correct_sent = train.correct_sent.map(lambda x: str(x))
# test.correct_sent = test.correct_sent.map(lambda x: str(x))
# dev.correct_sent = dev.correct_sent.map(lambda x: str(x))
# Combine datasets
train_all = pd.concat([train, clang8, relco, gera_train, gera_dev, my_data], ignore_index=True)
augmented_data = dataset.select(range(100000))
# Convert to Hugging Face Dataset format
fine_tune = {
    'train': Dataset.from_pandas(train_all),
    'test': Dataset.from_pandas(test),
    'dev': Dataset.from_pandas(dev)
}
fine_tune['train'] = datasets.concatenate_datasets([fine_tune['train'], augmented_data])
fine_tune

Generating train split: 10000000 examples [02:37, 63324.22 examples/s]


{'train': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 184947
 }),
 'test': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 5000
 }),
 'dev': Dataset({
     features: ['corrupt_sent', 'correct_sent'],
     num_rows: 2500
 })}

In [4]:
fine_tune['train'] = fine_tune['train'].shuffle()

In [5]:
def format_dataset(examples):
    return {
        "text": [
                f"<|im_start|>system\nТы учитель русского языка, который проверяет эссе своих учеников.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: {corrupt}\nИсправленное предложение:<|im_end|>\n<|im_start|>assistant\n{correct}<|im_end|>"
        for corrupt, correct in zip(examples['corrupt_sent'], examples['correct_sent'])
        ]
    }

fine_tune = {key: value.map(format_dataset, batched=True) for key, value in fine_tune.items()}


Map: 100%|██████████| 2500/2500 [00:00<00:00, 141747.35 examples/s]


In [7]:
for i in fine_tune['train'].take(1):
    print(i)

{'corrupt_sent': 'Глава минздрава Ливана Хамад Хасан в интервью телекналу Sky News Arabia сообщил , что после взрыва в порту Бейрута ситуация катастрофическая : больницы в з сто улице и Ее окрестностях переоплнены ранеными,  а врачи испытывают нехватку медицинских принадл Бежностей .', 'correct_sent': 'Глава минздрава Ливана Хамад Хасан в интервью телеканалу Sky News Arabia сообщил , что после взрыва в порту Бейрута ситуация катастрофическая : больницы в столице и ее окрестностях переполнены ранеными , а врачи испытывают нехватку медицинских принадлежностей . ', 'text': '<|im_start|>system\nТы учитель русского языка, который проверяет эссе своих учеников.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: Глава минздрава Ливана Хамад Хасан в интервью телекналу Sky News Arabia сообщил , что после взрыва в порту Бейрута ситуация катастрофическая : больницы в з сто улице и Ее окрестностях переоплнены ранеными,  а врачи испытывают нехватку медицинских принадл Бежностей .\

In [7]:
# Load model and tokenizer
# model_name = "RefalMachine/ruadapt_qwen2.5_3B_ext_u48_instruct_v4"
# model_name = 'Qwen/Qwen2-7B-Instruct'
# model_name = 'Qwen/Qwen3-4B'
# model_name = 'unsloth/Qwen3-4B-unsloth-bnb-4bit'
model_name = 'unsloth/Qwen3-14B-unsloth-bnb-4bit'
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# config = AutoConfig.from_pretrained(model_name)
model, tokenizer = FastLanguageModel.from_pretrained(model_name = model_name,
    max_seq_length = 512,  # Context length - can be longer, but uses more memory
    load_in_4bit = True,     # 4bit uses much less memory
    load_in_8bit = False,    # A bit more accurate, uses 2x memory
    full_finetuning = False)
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.float16,
# )

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     trust_remote_code=True,
#     low_cpu_mem_usage=True,
# ).to(device)
# model.config.use_cache = False

# LoRA configuration
def find_all_linear_names(model):
    cls = bnb.nn.Linear4bit
    lora_module_names = {name.split(".")[-1] for name, module in model.named_modules() if isinstance(module, cls)}
    lora_module_names.discard("lm_head")
    return list(lora_module_names)

target_modules = find_all_linear_names(model)
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,           # Choose any number > 0! Suggested 8, 16, 32, 64, 128
    target_modules = target_modules,
    lora_alpha = 64,  # Best to choose alpha = rank or rank*2
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = True,#dse or "unsloth" for very long context
    # random_state = 3407,
    use_rslora = True,   # We support rank stabilized LoRA
    # loftq_config = None,  # And LoftQ
)


# peft_config = LoraConfig(
#     lora_alpha=64,
#     target_modules=target_modules,
#     lora_dropout=0.05,
#     r=16,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-80GB. Num GPUs = 1. Max memory: 79.325 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Loading checkpoint shards: 100%|██████████| 3/3 [03:09<00:00, 63.08s/it]
Unsloth 2025.5.7 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


In [8]:
for i in fine_tune['train'].take(1):
    print(i)

{'corrupt_sent': 'Изд коронавируса автолизинг во всем мире сделали элитно езаболевания , опаснее которого чеовечесэво и не видывало .', 'correct_sent': 'Из коронавируса во всем мире сделали элитное заболевание , опаснее которого человечество и не видывало . ', 'text': '<|im_start|>system\nТы учитель русского языка, который проверяет эссе своих учеников.<|im_end|>\n<|im_start|>user\nИсправь ошибки в следующем предложении: Изд коронавируса автолизинг во всем мире сделали элитно езаболевания , опаснее которого чеовечесэво и не видывало .\nИсправленное предложение:<|im_end|>\n<|im_start|>assistant\nИз коронавируса во всем мире сделали элитное заболевание , опаснее которого человечество и не видывало . <|im_end|>'}


In [11]:
(len(fine_tune['train']) // 8 // 4 + 1)

5780

In [10]:
from transformers import EarlyStoppingCallback

In [12]:
args = SFTConfig(
    output_dir="qwen3-14b_rugec",
    eval_strategy="steps",
    eval_steps=100,
    logging_steps=10,
    learning_rate=1e-4,
    per_device_train_batch_size=8,  # Reduce batch size to prevent memory issues
    per_device_eval_batch_size=8,
    resume_from_checkpoint="last-checkpoint",
    weight_decay=0.02,
    gradient_accumulation_steps=4,
    # warmup_steps=200,
    save_total_limit=1,
    # num_train_epochs=1,
    max_steps=5800, 
    bf16=True,
    optim = "adamw_8bit",
    push_to_hub=True,
    hub_strategy="checkpoint",
    metric_for_best_model='loss',
    report_to = 'all',
    save_strategy='best',
    load_best_model_at_end=True,
    lr_scheduler_type="cosine", 
    warmup_ratio=0.01,
    dataset_text_field='text',
    max_seq_length=512,
    packing=False,
    # disable_tqdm=False
)


In [13]:
import logging
logging.getLogger("wandb").setLevel(logging.ERROR)

In [14]:
# Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=fine_tune['train'],
    eval_dataset=fine_tune['dev'],
    # peft_config=peft_config,
    processing_class=tokenizer,
    args=args,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=15)],
)

# Training
unsloth_train(trainer)


Map (num_proc=28): 100%|██████████| 2500/2500 [00:03<00:00, 729.63 examples/s]
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 184,947 | Num Epochs = 2 | Total steps = 5,800
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 4 x 1) = 32
 "-____-"     Trainable parameters = 128,450,560/14,000,000,000 (0.92% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Tracking run with wandb version 0.18.5
wandb: Run data is saved locally in /home/jupyter/work/resources/rugec/notebooks/qwen_gec/wandb/run-20250601_013133-ttjrdkt8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run qwen3-14b_rugec
wandb: ⭐️ View project at https://wandb.ai/mika5883/huggingface
wandb: 🚀 View

Unsloth: Will smartly offload gradients to save VRAM!


  0%|          | 10/5800 [00:53<7:07:03,  4.43s/it]

{'loss': 2.5817, 'grad_norm': 6.015710830688477, 'learning_rate': 1.5517241379310346e-05, 'epoch': 0.0}


  0%|          | 20/5800 [01:31<5:53:02,  3.66s/it]

...

100%|██████████| 5800/5800 [7:21:29<00:00,  3.52s/it]

{'loss': 0.7166, 'grad_norm': 0.7854896187782288, 'learning_rate': 7.483648256645382e-12, 'epoch': 1.0}



100%|█████████▉| 312/313 [01:23<00:00,  3.83it/s]
                                                     
100%|██████████| 5800/5800 [7:22:53<00:00,  4.58s/it]

{'eval_loss': 0.6499903202056885, 'eval_runtime': 83.8473, 'eval_samples_per_second': 29.816, 'eval_steps_per_second': 3.733, 'epoch': 1.0}
{'train_runtime': 26576.4904, 'train_samples_per_second': 6.984, 'train_steps_per_second': 0.218, 'train_loss': 0.8717163933556655, 'epoch': 1.0}


TrainOutput(global_step=5800, training_loss=0.8717163933556655, metrics={'train_runtime': 26576.4904, 'train_samples_per_second': 6.984, 'train_steps_per_second': 0.218, 'total_flos': 3.1927415068648243e+18, 'train_loss': 0.8717163933556655, 'epoch': 1.0034603572818894})

In [15]:
trainer.push_to_hub('first_qwen3_14b')

/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/other.py:762: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 0fd4626e-0d0c-429f-92f7-f06c7da79904)') - silently ignoring the lookup for the file config.json in unsloth/Qwen3-14B-unsloth-bnb-4bit.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/peft/utils/save_and_load.py:227: UserWarning: Could not find a config file in unsloth/Qwen3-14B-unsloth-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(
100%|██████████| 2/2 [00:03<00:00,  1.67s/it]


CommitInfo(commit_url='https://huggingface.co/mika5883/qwen3-14b_rugec/commit/c715e348c7788f6cdab5ac6c0bd73d3f496562e1', commit_message='first_qwen3_14b', commit_description='', oid='c715e348c7788f6cdab5ac6c0bd73d3f496562e1', pr_url=None, repo_url=RepoUrl('https://huggingface.co/mika5883/qwen3-14b_rugec', endpoint='https://huggingface.co', repo_type='model', repo_id='mika5883/qwen3-14b_rugec'), pr_revision=None, pr_num=None)